In [1]:
%pip install --upgrade pip

# Uninstall conflicting packages
%pip uninstall -y langchain-core langchain-openai langchain-experimental beautifulsoup4 langchain-community langchain chromadb beautifulsoup4

# Install compatible versions of langchain-core and langchain-openai
%pip install langchain-core==0.3.6
%pip install langchain-openai==0.2.1
%pip install langchain-experimental==0.3.2
%pip install langchain-community==0.3.1
%pip install langchain==0.3.1

# Install remaining packages
%pip install chromadb==0.5.11
%pip install beautifulsoup4==4.12.3

# Restart the kernel after installation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
Note: you may need to restart the kernel to use updated packages.
Found existing installation: langchain-core 0.2.43
Uninstalling langchain-core-0.2.43:
  Successfully uninstalled langchain-core-0.2.43
Found existing installation: langchain-openai 0.1.8
Uninstalling langchain-openai-0.1.8:
  Successfully uninstalled langchain-openai-0.1.8
Found existing installation: beautifulsoup4 4.13.4
Uninstalling beautifulsoup4-4.13.4:
  Successfully uninstalled beautifulsoup4-4.13.4
Found existing installation: langchain-community 0.2.5
Uninstalling langchain-community-0.2.5:
  Successfully uninstalled langchain-community-0.2.5
Found existing installation: langchain 0.2.5
Uninstalling langchain-0.2.5:
  Successfully uninstalled langchain-0.2.5
Note: you may need to restart the ker

In [2]:
# New OS parameter to avoid warnings.  
# This will not have a material impact on your code, but prevents warnings from appearing related to new LangChain features.
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker

In [ ]:
# OpenAI Setup
os.environ['OPENAI_API_KEY'] = ''
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
#### INDEXING ####

In [38]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://kbourne.github.io/chapter1.html",), 
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [39]:
# Split
text_splitter = SemanticChunker(OpenAIEmbeddings())
splits = text_splitter.split_documents(docs)

In [40]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
#### RETRIEVAL and GENERATION ####

In [41]:
# Prompt - ignore LangSmith warning, you will not need langsmith for this coding exercise
prompt = hub.pull("jclemens24/rag-prompt")

/opt/conda/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [42]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [43]:
# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [44]:
# Chain it all together with LangChain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [46]:
# Question - run the chain
rag_chain.invoke("How does RAG compare with fine-tuning?")

"RAG (Retrieval-Augmented Generation) and fine-tuning are two different approaches for enhancing the capabilities of large language models (LLMs). \n\nRAG allows models to access and utilize a vast amount of external data, such as company-specific information, customer interactions, and product details, without permanently altering the model itself. This means that RAG can leverage all available data to provide more relevant and context-aware responses, making it particularly useful for organizations with extensive internal data resources.\n\nIn contrast, fine-tuning involves adjusting the model's weights and biases based on new training data, which permanently changes how the model interacts with new inputs. Fine-tuning is more suitable for teaching the model specialized tasks or adapting it to a specific domain. However, it comes with limitations, such as context window sizes and the risk of overfitting on a specific dataset.\n\nIn summary, RAG focuses on utilizing external data dyna

input_variables=['context', 'question'] 

messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved-context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


"You are an assistant for question-answering tasks. Use the following pieces of retrieved-context to answer the question. If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Answer:"